In [ ]:
import numpy as np
import sys

sys.path.append('..')
from utils import load_diabetes, train_val_test_split

data_utils = load_diabetes()

data_utils.shape

data_list=[]

f = open("diabete.txt",encoding = "utf-8")
a_list=f.readlines()
f.close()
for line in a_list:
    line1=line.replace('\n', '')
    line2=list(line1.split(' '))
    y=float(line2[0])
    x= [float(line2[i].split(':')[1]) for i in (1,2,3,4,5,6,7,8)]
    data_list.append(x+[y])


data_array_1=np.array(data_list)[:,:-1]
data_array_0=np.ones((data_array_1.shape[0],1))
data_array_2=data_array_1*data_array_1
data_array_3=np.empty((data_array_1.shape[0],0))

for i in range(data_array_1.shape[1]):
    for j in range(data_array_1.shape[1]):
        if i<j:
            data_array_i=data_array_1[:,i]*data_array_1[:,j]
            data_array_i=np.reshape(data_array_i,(-1,1))
            data_array_3=np.hstack((data_array_3,data_array_i))

data_array_4=np.reshape(np.array(data_list)[:,-1],(-1,1))
data=np.hstack((data_array_0,data_array_1,data_array_2,data_array_3,data_array_4))

n_train = 500
n_val = 150

metrics = []
variables = []

hparams = {
    'gam': 5,
    'eta': 0.1,
    'alpha': 0.01,
    'beta': 0.01,
    'epsilon': 1e-5,
    'max_iters_outer': 100,
    'max_iters_inner': 100,
    'M': 1e-3,
    't': 1e-3
}

epochs = 80
plot_results = True

In [ ]:
c0, w0, b0, xi0 = 10 * np.ones(n_train), np.random.randn(45), np.random.randn(), np.random.randn(n_train)
from algorithms import barrier_blo
for seed in range(10):

    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(data, seed, n_train, n_val)
    datasets = {
        'x_train': x_train, 'y_train': y_train, 'x_val': x_val, 
        'y_val': y_val, 'x_test': x_test, 'y_test': y_test
    }
    
    problem = barrier_blo.SVM_Problem(datasets)
    algo = barrier_blo.Barrier_BLO(problem, hparams)
    
    # metrics_seed, variables_seed = barrier_blo(x_train, y_train, x_val, y_val, x_test, y_test, hparams, epochs)
    metrics_seed, c, w, b, xi, grad_norm = algo.upper_loop(c0, w0, b0, xi0, hparams)
    metrics.append(metrics_seed)
    # variables.append([c, w, b, xi])

train_acc = np.array([[x['train_acc'] for x in metrics] for metrics in metrics])
val_acc = np.array([[x['val_acc'] for x in metrics] for metrics in metrics])
test_acc = np.array([[x['test_acc'] for x in metrics] for metrics in metrics])
val_loss = np.array([[x['val_loss'] for x in metrics] for metrics in metrics])
test_loss = np.array([[x['test_loss'] for x in metrics] for metrics in metrics])

time_computation = np.array([[x['time_computation'] for x in metrics] for metrics in metrics])